### LIBRARIES

In [1]:
!pip install sentence-transformers

In [80]:
!pip install tensorflow

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/385.0 MB 16.2 MB/s eta 0:00:24
   ---------------------------------------- 1.2/385.0 MB 18.4 MB/s eta 0:00:21
   ---------------------------------------- 2.5/385.0 MB 19.9 MB/s eta 0:00:20
   ---------------------------------------- 4.2/385.0 MB 24.3 MB/s eta 0:00:16
    --------------------------------------- 5.6/385.0 MB 27.7 MB/s eta 0:00:14
    --------------------------------------- 7.8/385.0 MB 31.2 MB/s eta 0:00:13
    --------------------------------------- 9.5/385.0 MB 32.1 MB/s eta 0:00:12
   - -------------------------------------- 11.9/385.0 MB 40.9 MB/s eta 0:00:10
   - -------------------------------------- 15.1/385.0 MB 50.4 MB/s eta 0:00:08
   - -------------------------------------- 18.2/385.0 MB 59.5 M

In [81]:
import pandas as pd
import numpy as np
import re
import ast
import unicodedata
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.metrics import edit_distance
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100000)

### DATA IMPORT

In [3]:
df1 = pd.read_csv('data/datasetia_full_1.csv', index_col = 0)
df2 = pd.read_csv('data/datasetia_full_2.csv', index_col = 0)
df3 = pd.read_csv('data/datasetia_full_3.csv', index_col = 0)
df4 = pd.read_csv('data/datasetia_full_3_4.csv', index_col = 0)
df5 = pd.read_csv('data/datasetia_full_5.csv', index_col = 0)

In [4]:
# Unimos los dos archivos
df = pd.concat([df1, df2, df3, df4, df5], axis=0).reset_index(drop=True)

In [5]:
print(f'df_1: {len(df1)}')
print(f'df_2: {len(df2)}')
print(f'df_3: {len(df3)}')
print(f'df_4: {len(df4)}')
print(f'df_5: {len(df5)}')
print('-----------')
print(f'df: {len(df)}')

df_1: 2556
df_2: 2556
df_3: 2557
df_4: 5090
df_5: 2535
-----------
df: 15294


# PARTE 1: HERRAMIENTA DE AYUDA EN LA SIMILITUD DE MARCAS

### DATA PRE-PROCESSING

En primer lugar, hacemos una primera visualización de los datos para ver que estructura tiene nuestro *dataframe*.

In [6]:
df.sample(5)

,numero_expediente,resolucion,numero_de_resolución,denominacion,vigencia,titular,clase,gaceta,tipo,fecha_solicitud,fecha_resolucion,nombre_opositor,signo_opositor_opositores,argumento_oposición,explicacion_argumentos_oposicion,resolucion_organismo
5010,SD2021/0080419,aprobada_sin_oposicion,17413,Enzoimmune Active Immune Modulator,08.06.2031,Rosetta Lifecare Bulgaria,5,939,Mixta,26 de agosto de 2021,31 de marzo de 2022,NaN,NaN,NaN,NaN,Conceder el registro de la Marca Enzoimmune A...
8241,SD2022/0001149,aprobada_sin_oposicion,56790,VOLVIDIS,16.11.2031,SYNGENTA CROP PROTECTION AG,5,960.0,Nominativa,6 de enero de 2022,24 de agosto de 2022,NaN,NaN,NaN,NaN,Conceder el registro de la Marca VOLVIDIS (No...
3397,SD2018/0067789,aprobada_sin_oposicion,706,PLINAZOLIN,30 de mayo de 2028,Syngenta Participations AG,5,NaN,Nominativa,29/05/2018,26 de febrero de 2019,NaN,NaN,NaN,NaN,Conceder el registro de la Marca PLINAZOLIN (N...
15001,SD2022/0099844,negada_sin_oposicion,6278,PDPAOLA,NaN,SASMAT RETAIL S.L.,14,980.0,Nominativa,29 de septiembre de 2022,29 de febrero de 2024,"PAOLACALZADOS PABLO, S.L.",PAOLA,Artículo 136 literal a) de la Decisión 486 de ...,El signo solicitado (PDPAOLA) es similar a la ...,Negar el registro de la Marca PDPAOLA (Nominat...
3208,SD2018/0058851,aprobada_sin_oposicion,27993,CTL (Nominativa),30.04.2028,"Crisis Text Line, Inc.","[38, 45]",NaN,Nominativa,NaN,15 de julio de 2019,NaN,NaN,NaN,NaN,Conceder el registro de la Marca CTL (Nominati...


Las columnas que nos van a interesar para el desarrollo del modelo contienen la siguiente información:

- **numero_expediente**: identificador de la solicitud por parte de una empresa para registrar una marca
- **resolucion**: resultado de la solicitud.
- **denominacion**: nombre de la marca que se quiere registrar
- **titular**: nombre de la empresa que quiere registrar la marca
- **clase**: categoría de la marca que se quiere registrar
- **tipo**: tipo de registro que se quiere hacer (nombre, imágen o las dos)
- **signo_opositor_opositores**: marca que hace oposición a la que se intenta registrar
- **nombre_opositor**: empresa dueña de la marca que hace oposicion

Vemos el tipo de dato que presenta cada columna.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15294 entries, 0 to 15293
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   numero_expediente                 15294 non-null  object
 1   resolucion                        15294 non-null  object
 2   numero_de_resolución              15294 non-null  int64 
 3   denominacion                      14623 non-null  object
 4   vigencia                          10585 non-null  object
 5   titular                           15293 non-null  object
 6   clase                             15293 non-null  object
 7   gaceta                            11569 non-null  object
 8   tipo                              15293 non-null  object
 9   fecha_solicitud                   13097 non-null  object
 10  fecha_resolucion                  15287 non-null  object
 11  nombre_opositor                   2934 non-null   object
 12  signo_opositor_opo

De la primera visualización de los datos, podemos ver como hay 3 columnas: *vigencia*, *fecha_solicitud* y *fecha_resolucion* que, haciendo referencia a fechas, son de tipo *object* y, además, presentan formatos diferentes entre sí. Vamos a transformar las columnas a tipo *datetime* y homogeneizar el formato entre ellas.

El formato más conflictivo vendrá de los campos de tipo '02 de junio de 2023', '02 junio de 2023' o '2 jun. 2023', ya que pandas no reconoce el nombre del mes en español. Para ello, creamos un diccionario donde convirtamos el mes de una palabra a un número. Posteriormente, con el uso de expresiones regulares, buscamos los resgistros con ese formato y lo modificamos al formato *año-mes-dia*.

In [8]:
# Diccionario para los nombres de los meses en español
meses_espanol = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06', 
    'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12',
    'ene.': '01', 'feb.': '02', 'mar.': '03', 'abr.': '04', 'may.': '05', 'jun.': '06', 
    'jul.': '07', 'ago.': '08', 'sep.': '09', 'oct.': '10', 'nov.': '11', 'dic.': '12',
    'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may': '05', 'jun': '06',
    'jul': '07', 'ago': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dic': '12',
}

# Expresiones regulares para formatear los registros de tipo '02 de junio de 2023'
def procesar_fecha_espanol(texto):
    # Primero pasamos todo a minúscula y eliminamos espacios 
    texto = texto.lower().strip()

    # A veces el ultimo caracter e sun punto, si es asi lo eliminamos
    if texto[len(texto)-1] == '.':
        texto = texto[:-1]

    # Otras veces hay un punto en algun lugar del campo. Los ustituimos por un espacio en blanco
    for w in texto:
        if w=='.' or w=='º':
            texto = texto.replace(w, ' ')

    
    # Buscar fechas en formato 'dd de mes de yyyy' o 'd de mes de yyyy'
    match = re.search(r'(\d{1,2})\s+(?:de\s+)?([enero|febrero|marzo|abril|mayo|junio|julio|sgosto|septiembre|octubre|noviembre|diciembre]|[a-zA-Záéíóúñ]{3,}\.?\s*|[a-zA-Záéíóúñ]+)\s+(?:del\s+|de\s+)?(\d{4})', texto) 
    # \d representa cualquier digito del 1 al 9. 
    # {1,2} representa uno o dos digitos. 
    # \s representa espacio en blanco. 
    # ? representa que es un bloque opcional
    # con el uso de () en la expresion regular anterior podemos definir grupos. En este caso usaremos el grupo dia, mes y año para representar estos valores
    if match:
        dia = match.group(1).strip()
        mes = match.group(2).strip()
        año = match.group(3).strip()
        if 'de' in dia or 'de' in mes or 'de' in año:
            match2 = re.search(r'(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+(\d{1,2})\s+de\s+(\d{4})', texto)
            if match2:
                dia = match2.group(2).strip()
                mes = match2.group(1).strip()
                año = match2.group(3).strip()
        if mes in meses_espanol: # convertimos de palabra a número
            mes_num = meses_espanol[mes]
            # Formatear la fecha en formato numérico estándar
            return f'{año}-{mes_num}-{int(dia):02d}'
    
    # Si no se encuentra el formato esperado, devuelve el texto sin cambios
    return texto

# Función para intentar convertir la fecha a un formato estándar 
def parse_date(date):
    # Si el valor es NaN no hace nada
    if pd.isna(date):
        return date
    
    # Procesar primero las fechas con meses en español
    date_procesada = procesar_fecha_espanol(date)
    
    # Intentamos convertir cualquier formato restante a formato fecha
    try:
        return pd.to_datetime(date_procesada, dayfirst=True, errors='coerce')
    except Exception as e:
        return pd.NaT

# Aplicamos la funcion a las columnas que nos interesan
df['vigencia_formated'] = df['vigencia'].apply(parse_date)
df['fecha_solicitud_formated'] = df['fecha_solicitud'].apply(parse_date)
df['fecha_resolucion_formated'] = df['fecha_resolucion'].apply(parse_date)


Comprobamos que se ha realizado correctamente. Para cada una de las columnas modificadas vamos a comparar los valores nulos de la nueva columna (terminada en *_formated*) con los valores anteriores.

In [9]:
# Todo OK ya que nos que son null son por no ser fecha
df_vi = df[df['vigencia_formated'].isna() & ~df['vigencia'].isna()][['vigencia', 'vigencia_formated']]
df_vi['vigencia'].value_counts()[:5]

vigencia
No aplica                     320
No concedida                   47
Denegada                       37
No aplica, registro negado     31
.                              14
Name: count, dtype: int64

En este caso vemos que los valores que no se han capturado es, o por error en los formatos (como por ejemplo *29 de junino de 2027*), o por no presentar un patrón similar a la expresión regular (como por ejemplo *diez años desde la fecha del documento*). Para el modelo que se presenta en este *notebook* no se van a utilizar las fechas, de modo que no se va a depurar. Más adelante, para futuras implementaciones, se modificará oportunamente.

In [10]:
df_fs = df[df['fecha_solicitud_formated'].isna() & ~df['fecha_solicitud'].isna()][['fecha_solicitud', 'fecha_solicitud_formated']]
df_fs['fecha_solicitud'].value_counts()[:5]

fecha_solicitud
No encontrado                  6
No se encuentra en el texto    5
No se especifica               2
No encontrado en el texto      1
No disponible en el texto      1
Name: count, dtype: int64

In [11]:
df[df['fecha_resolucion_formated'].isna() & ~df['fecha_resolucion'].isna()][['fecha_resolucion', 'fecha_resolucion_formated']]

,fecha_resolucion,fecha_resolucion_formated
671,de de 2016,NaT


Vemos que esta casi todo bien ya que los únicos valores de a nueva columna que son *null* son por valores que no son fecha. Por ello, eliminamos las columnas antiguas y renombramos las nuevas.

In [12]:
df.drop(['vigencia', 'fecha_solicitud', 'fecha_resolucion'], axis=1, inplace=True)
df.rename(columns={'vigencia_formated': 'vigencia', 'fecha_solicitud_formated': 'fecha_solicitud', 'fecha_resolucion_formated': 'fecha_resolucion'}, inplace=True)

Otra columna que nos va a ser de gran utilidad es *clase*. Esta columna es de tipo *string* y recoge todas las clases en las que esta registrada una marca. Es necesario convertirla de *string* a lista. Primero de todo modificaremos el formato para que sea comun a todos los registros y, posteriormente, la convertiremos a formato lista.

In [13]:
# Primero eliminamos aquellos registros que sean nulos ya que no nos van a servir para clasificar y luego modificamos los demas formatos
df = df.dropna(subset=['clase']).reset_index(drop=True)

for pos in range(0, len(df['clase']), 1):
    elem = df['clase'].iloc[pos].strip()
    if ~elem.startswith('[') == -1: # los que no comiencen por '[' los forzamos a que lo hagan
        elem = '[' + elem
    if ~elem.endswith(']') == -1: # igual para las terminaciones
        elem = elem + ']'
    if 'y' in elem:
        elem = elem.replace('y', ',') # sustituimos el valor final de 'y' en algunos registros por ',' para dar el formato lista
    if "'" in elem:
        elem = elem.replace("'", '') # eliminamos las comillas de string para que todo sea formato numerico
    if '\n' in elem:
        elem = elem.replace('\n', ',') # eliminamos el caracter '\n' por una coma de separador

    df['clase'].iloc[pos] = elem


In [14]:
# Evalua el contenido y lo transforma a lista
def convertir_a_lista(valor):
    if isinstance(valor, int):  # Verifica si es un entero
        return [valor]  # Convierte el entero a una lista
    elif isinstance(valor, str):  # Verifica si es una cadena
        try:
            # Intentar convertir la cadena a una lista
            return ast.literal_eval(valor)
        except (ValueError, SyntaxError):
            # Si hay un error, devolver None
            return None
    return valor  # Devuelve el valor original si no es ni entero ni caden

# Aplicar la conversión a la columna
df['clase'] = df['clase'].apply(convertir_a_lista)
df = df.dropna(subset=['clase']).reset_index(drop=True)

In [15]:
# Comprobamos que se ha hecho bien el cambio
type(df['clase'][0])

list

También podemos observar, de la primera visualización, que para la columna *resolucion*, existen casos que se escriben con tilde y otros sin. Es decir, hay valores que son *aprobada_con_oposicion* y otros que son *aprobada_con_oposición*. Para evitar errores en el filtrado posterior, vamos a eliminar todas las tildes

In [16]:
# Función para eliminar tildes
def quitar_tildes(texto):
    if pd.isna(texto):  # Verificar si es NaN o None
        return texto  # Retornar el valor tal cual
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

# Aplicamos la función a la columna 'resolucion'
df['resolucion'] = df['resolucion'].apply(quitar_tildes)

In [17]:
df[['resolucion']].groupby(['resolucion']).value_counts()

resolucion
aprobada_con_oposicion     566
aprobada_sin_oposicion    9559
archivada                    1
archivado                    1
denegada_con_oposicion       1
desistimiento                1
negada_con_oposicion       987
negada_sin_oposicion      4166
Name: count, dtype: int64

Vemos que hay 3 nuevas clases: *archivada*, *desistimmiento* y *denegada_con_oposicion*. Esta última la transformaremos a *negada_con_oposicion*, mientras que las dos primeras las eliminaremos ya que no nos aportan valor.

In [18]:
# Filtrar filas donde la columna Resolución no contenga 'archivada', 'archivado', 'desistimiento'
df = df[~df['resolucion'].isin(['archivada', 'archivado', 'desistimiento'])]

# Reemplazar 'denegada_con_oposicion' por 'negada_con_oposicion' en la columna Resolución
df['resolucion'] = df['resolucion'].replace('denegada_con_oposicion', 'negada_con_oposicion')

In [19]:
# oOmprobamos que esta OK
df[['resolucion']].groupby(['resolucion']).value_counts()

resolucion
aprobada_con_oposicion     566
aprobada_sin_oposicion    9559
negada_con_oposicion       988
negada_sin_oposicion      4166
Name: count, dtype: int64

Vamos a ver ahora los valores nulos que hay y si tienen sentido

In [20]:
df.isna().sum()

numero_expediente                       0
resolucion                              0
numero_de_resolución                    0
denominacion                          671
titular                                 1
clase                                   0
gaceta                               3720
tipo                                    1
nombre_opositor                     12355
signo_opositor_opositores           10137
argumento_oposición                 10059
explicacion_argumentos_oposicion    10060
resolucion_organismo                   25
vigencia                             5214
fecha_solicitud                      2213
fecha_resolucion                        8
dtype: int64

Puede ser que algunos valores, que hacen referencia a la oposición de registro, tengan sentido. Esto es debido a que algunas de las solicitudes son aprobadas o negadas sin oposición. Vamos a comprobar que los valores nulos para de las columnas *nombre_opositor*, *signo_opositor_opositores*, *argumento_oposición* y *explicacion_argumentos_oposicion* se correspondan con una resolución sin oposición.

In [21]:
lista_resolucion = ['aprobada_sin_oposicion', 'negada_sin_oposicion']

print(f"nombre_opositor: {len(df[df['nombre_opositor'].isna() & ~df['resolucion'].isin(lista_resolucion)])}")
print(f"signo_opositor_opositores: {len(df[df['signo_opositor_opositores'].isna() & ~df['resolucion'].isin(lista_resolucion)])}")
print(f"argumento_oposición: {len(df[df['argumento_oposición'].isna() & ~df['resolucion'].isin(lista_resolucion)])}")
print(f"explicacion_argumentos_oposicion: {len(df[df['explicacion_argumentos_oposicion'].isna() & ~df['resolucion'].isin(lista_resolucion)])}")


nombre_opositor: 2
signo_opositor_opositores: 30
argumento_oposición: 5
explicacion_argumentos_oposicion: 5


Para el desarrollo del modelo nos vamos a centrar principalmemnte en los casos en los que se ha hecho oposición (*aprobado_con_oposicion* y *negada_con_oposicion), que son precisamente el número de casos que se ve en la celda anterior. El resto de casos que toman el valor *nan* se deben a que son casos en los que no se ha hecho oposición y, como esas celdas hacen referencia a la oposición, están como *nan*. Como mencionabamos, son muy pocos los casos en los que hay valores faltantes de modo que lo dejaremos como está y los filtraremos en la parte de modelado.

### DATA PROCESSING

Para el registro de nuevas marcas influyen dos factores fundamentales:

- **denominacion**: Es el nombre de la marca que se intenta registrar
- **clase**: Categoría de negocios (por ejemplo, restauración, retail, etc.)

Cuando una emprese intente registrar un nuevo nombre (*denominacion*) tiene que indicar, además del nombre, la clase en la que lo desea hacer. Por ello, los parametros de entrada serán estos dos: **nombre** y **clase**. 

Una vez los tengamos recogidos, filtraremos el dataset por la clase indicada por el usuario (de momento una única clase, aunque en líneas futuras se abrirá el registro a un mayor número). En este punto vamos a comprobar varias cosas:

1. Que la marca que se intenta registrar no lo esté ya. Si es así directamente **no se podrá registrar**.
2. En caso de que no esté registrada buscaremos, dentro de esa misma clase, otras **marcas similares**. Esto lo haremos mediante dos métodos:

    a. **Parecido semántico**: Marcas que no se escriban igual pero tengan significados similares (por ejemplo: burger king, hamburguesas queen)
    
    b. **Parecido de caracteres**: Marcas que se escriban similar.

    Sumado a estas dos comparaciones se dirá en la medida que las palabras se parecen (terminos percentuales).
3. Por último, para la clase indicada por el usuario, se dará un **valor máximo y medio de similitud por clase** para aquellas solicitudes aprobadas. De esta forma el usuario podrá valorar mejor la magnitud de la similitud de la marca a registrar y tendrá un mayor **contexto** sobre si quiere iniciar el proceso o no

In [22]:
nombre = str(input('Introduzca el nombre de la clase que desea registrar: '))
clase = int(input('Introduzca la clase en la que desea registrar la marcas: '))

**PASO 1**: Filtramos el dataset por aquellos registros que contengan la clase donde quiero registrar la marca

In [23]:
df_flt = df[df['clase'].apply(lambda x: clase in x)].reset_index(drop=True)

**PASO 2**: Almaceno en una lista el nombre de todas las marcas que ya estan actualmente registradas en esa clase

In [24]:
marca_lst = [df_flt['denominacion'][pos] for pos in range(0, len(df_flt), 1) if (any('aprobada' in registro for registro in df_flt['resolucion']) and pd.notna(df_flt['denominacion'][pos]) and df_flt['denominacion'][pos] is not None)]

**PASO 3**: Buscamos la similitud semántica y de caracteres entre el nombre que queremos registrar y los que ya estan registrados.

**A. Similitud Semántica**: Vamos a utilizar el modelo preentrenado *Sentence-BERT*. De esta forma generamos *embeddings* de los nombres de las empresas y calculamos la similitud del coseno.

In [25]:
modelo = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Usamos embeddings ya que son las representaciones en forma vectorial del significado de una palabra. Asi podemos hacer la comparación semántica
# Obtenemos los embeddings de las empresas en la lista que hemos obtenido anteriormente
embeddings_sic = modelo.encode(marca_lst)

# Obtenemos el embedding de la nueva empresa
embedding_nueva_marca = modelo.encode([nombre])

# Calculamos la similitud de coseno entre la nueva empresa y cada empresa de la lista
# USamos la distancia del coseno porque mide el ángulo entre dos vectores (embeddings)
similitudes = cosine_similarity(embedding_nueva_marca, embeddings_sic)[0] 


**B. Similitud de Caracteres**: Vamos a hacer una comparación de los caracteres que conforman una palabra. Buscamos alabras que, aunque tengan significados totalmente diferente, se escriban similar. Para ello vamos a usar la *distancia de Levenshtein*, que mide el porcentaje de similitud basado en el número de cambios necesarios para convertir una palabra en otra. Cuanto más bajo sea el número, mayor es la similitud.

In [26]:
# Vamos a ir guardando en un array las distancias entre la nueva marca y las que ya hay en la BBDD
distancias_levenshtein = []

for empresa in marca_lst:
    lev_dist = edit_distance(nombre, empresa) # Calculamos la distancia levenshtein entre la nueva marca y cada una de las que hay en la BBDD
    max_len = max(len(nombre), len(empresa)) # Obtiene la longitud de la cadena más larga entre las dos empresas. Esto es importante porque la similitud se mide en relación con la longitud de las palabras.
    similitud = (1 - lev_dist / max_len)  # Similitud en porcentaje
    distancias_levenshtein.append(similitud)

**PASO 4**: Para coger mayor perspectiva de como de grandes o pequeños son esos porcentajes vamos a dar contexto a la situación. Para ello, para las solicutdes aprobadas con oposición, daremos el valor medio y máximo de similitud con la que se han aprobado. Haremos lo mismo para las negadas con oposición para su valor mínimo.

**A. Aprobadas con oposicion**: Para las solicitudes que han sido aprobadas con oposicion sacamos, del par *denominacion-signo_opositor_opositores* los *embeddings* y sacamos el valor promedio y el máximo. La idea, como se ha comentado antes, es tener una visión de cuál es el valor a partir del cuál se han aprobado las solicitudes.

In [27]:
# Para el dataset ya filtrado por la clase, filtro otra vez por la resolución
df_flt_aprob = df_flt[df_flt['resolucion']=='aprobada_con_oposicion'].reset_index()

# Vamos a ir almacenando en una lista el conjunto de dupas denominación-signo_opositor_opositores
list_sim_aprob = []
marca_sim_aprob = [(df_flt_aprob['denominacion'][pos], [n.strip() for n in df_flt_aprob['signo_opositor_opositores'][pos].split(',')]) for pos in range(0, len(df_flt_aprob), 1) if pd.notna(df_flt_aprob['denominacion'][pos]) and pd.notna(df_flt_aprob['signo_opositor_opositores'][pos]) and df_flt_aprob['denominacion'][pos] is not None]

# Calculamos la similitud semantica entre los pares
modelo = SentenceTransformer('paraphrase-MiniLM-L6-v2')
sim_aprob = []
for m1, m2 in marca_sim_aprob:
    embeddings_m1 = modelo.encode(m1).reshape(1, -1)
    if type(m2)==list:
        m2_list = []
        for m in m2:
            m2_list.append(modelo.encode(m).reshape(1, -1))
        for emb in m2_list:
            sim_aprob.append((m1, m, cosine_similarity(embeddings_m1, emb)[0][0]))
    else:
        embeddings_m2 = modelo.encode(m2).reshape(1, -1)   
        sim_aprob.append((m1, m2, cosine_similarity(embeddings_m1, embeddings_m2)[0][0]))

# Calculamos la similitud de caracteres
distancias_lev_aprob = []
for m1, m2 in marca_sim_aprob:
    if type(m2)==list:
        for m in m2:
            lev_dist_aprob = edit_distance(m1, m)
            max_len_aprob = max(len(m1), len(m)) 
            similitud_aprob = (1 - lev_dist_aprob / max_len_aprob)  
            distancias_lev_aprob.append((m1, m, similitud_aprob))
    else:
            distancias_lev_aprob.append((m1, m2, similitud_aprob))

# Sacamos valores máximos y medios
v_medio_aprob_sem = np.mean([value[2] for value in sim_aprob])
v_max_aprob_sem = max([value[2] for value in sim_aprob])
v_medio_aprob_car = np.mean([value[2] for value in distancias_lev_aprob])
v_max_aprob_car = max([value[2] for value in distancias_lev_aprob])

**B. Negadas con oposicion**: Para las solicitudes que han sido negadas con oposicion sacamos, del par *denominacion-signo_opositor_opositores* los *embeddings* y 
sacamos el valor promedio y el mínimo. La idea, como se ha comentado antes, es tener una visión de cuál es el valor a partir del cuál se han denegado las solicitudes.

In [28]:

# Para el dataset ya filtrado por la clase, filtro otra vez por la resolución
df_flt_nega = df_flt[df_flt['resolucion']=='negada_con_oposicion'].reset_index()

# Vamos a ir almacenando en una lista el conjunto de dupas denominación-signo_opositor_opositores
list_sim_nega = []
marca_sim_nega = [(df_flt_nega['denominacion'][pos], [n.strip() for n in df_flt_nega['signo_opositor_opositores'][pos].split(',')]) for pos in range(0, len(df_flt_nega), 1) if pd.notna(df_flt_nega['denominacion'][pos]) and pd.notna(df_flt_nega['signo_opositor_opositores'][pos]) and df_flt_nega['denominacion'][pos] is not None]

# Calculamos la similitud semantica entre los pares
modelo = SentenceTransformer('paraphrase-MiniLM-L6-v2')
sim_nega = []
for m1, m2 in marca_sim_nega:
    embeddings_m1 = modelo.encode(m1).reshape(1, -1)
    if type(m2)==list:
        m2_list = []
        for m in m2:
            m2_list.append(modelo.encode(m).reshape(1, -1))
        for emb in m2_list:
            sim_nega.append((m1, m, cosine_similarity(embeddings_m1, emb)[0][0]))
    else:
        embeddings_m2 = modelo.encode(m2).reshape(1, -1)   
        sim_nega.append((m1, m2, cosine_similarity(embeddings_m1, embeddings_m2)[0][0]))

# Calculamos la similitud de caracteres
distancias_lev_nega = []
for m1, m2 in marca_sim_nega:
    if type(m2)==list:
        for m in m2:
            lev_dist_nega = edit_distance(m1, m)
            max_len_nega = max(len(m1), len(m)) 
            similitud_nega = (1 - lev_dist_nega / max_len_nega)  
            distancias_lev_nega.append((m1, m, similitud_nega))
    else:
            distancias_lev_nega.append((m1, m2, similitud_nega))

# Sacamos valores máximos y medios
v_medio_nega_sem = np.mean([value[2] for value in sim_nega])
v_min_nega_sem = min([value[2] for value in sim_nega])
v_medio_nega_car = np.mean([value[2] for value in distancias_lev_nega])
v_min_nega_car = min([value[2] for value in distancias_lev_nega])

### OUTPUT

In [29]:
if nombre in marca_lst:
    mensaje_1 = f'Lamentablemente, "{nombre}" ya está registrado dentro de esta clase.'
else:
    mensaje_1 = f'Actualmente, no existe ninguna marca registrada como "{nombre}" dentro de esta clase.'

In [30]:
prompt = f"""
Estas tratando de registrat la marca "{nombre}" dentro de la clase {clase}.
{mensaje_1}

En esta clase hay registradas {len(marca_lst)} marcas. De ellas, para la marca que intenta registrar:
- Hay un {max(similitudes):.2%} de similitud semántica con la marca {marca_lst[np.argmax(similitudes)]} ya registrada.
- Hay un {max(distancias_levenshtein):.2%} de similitud de caracteres con la marca {marca_lst[np.argmax(distancias_levenshtein)]} ya registrada.

Para esta clase, de todas las solicitudes que se han aprobado con oposición:
- El valor medio de similitud semántica ha sido {v_medio_aprob_sem:.2%}, mientras que el valor máximo es del {v_max_aprob_sem:.2%}
- El valor medio de similitud de caracteres ha sido {v_medio_aprob_car:.2%}, mientras que el valor máximo es del {v_max_aprob_car:.2%}

Para las solicitudes que se han negado con oposición:
- El valor medio de similitud semántica ha sido {v_medio_nega_sem:.2%}, mientras que el valor mínimo es del {v_min_nega_sem:.2%}
- El valor medio de similitud de caracteres ha sido {v_medio_nega_car:.2%}, mientras que el valor mínimo es del {v_min_nega_car:.2%}
"""

In [31]:
print(prompt)


Estas tratando de registrat la marca "OPERA" dentro de la clase 9.
Actualmente, no existe ninguna marca registrada como "OPERA" dentro de esta clase.

En esta clase hay registradas 3825 marcas. De ellas, para la marca que intenta registrar:
- Hay un 52.27% de similitud semántica con la marca ARIANE ya registrada.
- Hay un 80.00% de similitud de caracteres con la marca OPERR ya registrada.

Para esta clase, de todas las solicitudes que se han aprobado con oposición:
- El valor medio de similitud semántica ha sido 38.44%, mientras que el valor máximo es del 100.00%
- El valor medio de similitud de caracteres ha sido 31.19%, mientras que el valor máximo es del 100.00%

Para las solicitudes que se han negado con oposición:
- El valor medio de similitud semántica ha sido 43.87%, mientras que el valor mínimo es del -7.62%
- El valor medio de similitud de caracteres ha sido 30.46%, mientras que el valor mínimo es del 0.00%



# PARTE 2: MODELO DE PREDICCIÓN

In [32]:
dataset = df[['resolucion', 'denominacion', 'clase', 'nombre_opositor']]

In [33]:
dataset.sample(5)

,resolucion,denominacion,clase,nombre_opositor
13186,negada_sin_oposicion,OCTOPUS,"[9, 17]",AMERICA TOWERTECH AMERICAS LTDA.
1019,aprobada_sin_oposicion,NaN,[1],NaN
7984,aprobada_sin_oposicion,OPTICARE,"[10, 20]",NaN
5099,aprobada_sin_oposicion,ACCUPOINT,[8],NaN
581,aprobada_sin_oposicion,ORIGINAL NEW YORK SELTZER (Nominativa),[32],NaN


Vemos a ver los nulos que hay en cada columna

In [34]:
dataset.isna().sum()

resolucion             0
denominacion         671
clase                  0
nombre_opositor    12355
dtype: int64

En los casos en los que oposicion sea *NaN* los eliminamos ya que no aportan valor. En el caso de *nombre_opositor* los sustituimos por 0.

In [35]:
# Eliminamos en denominacion
dataset = dataset.dropna(subset=['denominacion'])

# Sustituimos por 0 en nombre_opositor
dataset['nombre_opositor'] = dataset['nombre_opositor'].fillna(0)

In [36]:
# Comprobamos que se haya hecho bien
dataset.isna().sum()

resolucion         0
denominacion       0
clase              0
nombre_opositor    0
dtype: int64

Lo rimero que tenemos que hacer es transformar la columna *target* en valores numéricos. Para ello mapeamos los valores


In [37]:
dataset['resolucion'] = dataset['resolucion'].replace({'aprobada_con_oposicion': 1, 'aprobada_sin_oposicion': 1, 
                                                       'negada_con_oposicion': 0, 'negada_sin_oposicion': 0
})

Para las variables *denominacion* y *nombre_opositor* realizamos un *One Hot Encoding*

In [38]:

dataset_encoded = pd.get_dummies(dataset, columns=['denominacion', 'nombre_opositor'], dtype=int)

Por último, como la variable clase es una lista de elementos, expandimos la lista para que cada valor este en una columna

In [39]:
# Transformar la columna 'clase' directamente en el DataFrame original
df_clase = dataset_encoded['clase'].apply(pd.Series)

# Renombrar las columnas para que tengan un formato más descriptivo (opcional)
df_clase.columns = [f'clase_{i+1}' for i in range(df_clase.shape[1])]

# Asignar las nuevas columnas al DataFrame original
for col in df_clase.columns:
    dataset_encoded[col] = df_clase[col]

# Eliminar la columna original 'clase'
dataset_encoded.drop(columns=['clase'], inplace=True)

Como ahora, tras haber pasado las listas a multiples columnas se habrán generado valores *Na*, los sustituimos por 0.

In [40]:
dataset_encoded.fillna(0, inplace=True)

In [41]:
dataset_encoded.isna().sum().sum()

0

Una vez teemos el dataset, dividimos entre X e y 

In [42]:
X = dataset_encoded.drop('resolucion', axis=1)
y = dataset_encoded['resolucion']

Dividimos entre *train* y *test*

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1903)

In [44]:
# Vemos dimensiones
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

X_train: (10225, 13797)
X_test: (4383, 13797)
y_train: (10225,)
y_test: (4383,)


Vamos a comprobar también si las clases están balanceadas

In [45]:
y_train.value_counts()

resolucion
1    6690
0    3535
Name: count, dtype: int64

Como no lo estan aplicamos *SMOTE*

In [46]:
# Aplicar PCA para reducir a 100 componentes principales
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Ahora aplica SMOTE en los datos reducidos
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


Vemos los resultados tras el balanceo

In [47]:
y_train.value_counts()

resolucion
0    6690
1    6690
Name: count, dtype: int64

Para la parte de modelado vamos a emplear diferentes modelos y evaluar cual funciona mejor. Los modelos que se van a probar son:

- **Regresión logística**
- **Árbol de decisión**
- **Random Forest**
- **Red Neuronal**

Como vamos a aplicar un *Grid Search* para afinar bien el modelo, definimos un diccionario con los parámetros en función del modelo.

In [48]:
param_grids = {
    "Logistic Regression": {
        'penalty': ['l2'],  # Regresión con regularización L2
        'C': [0.01, 0.1, 1, 10],  # Inversa de la fuerza de regularización
        'solver': ['lbfgs'],  # Optimizador
        'max_iter': [100, 200]  # Número de iteraciones
    },
    "Decision Tree": {
        'criterion': ['gini', 'entropy'],  # Función para medir la calidad del split
        'max_depth': [None, 10, 20],  # Profundidad máxima del árbol
        'min_samples_split': [2, 10, 20],  # Mínimo número de muestras para un split
        'min_samples_leaf': [1, 5, 10]  # Mínimo número de muestras por hoja
    },
    "Random Forest": {
        'n_estimators': [100, 200],  # Número de árboles en el bosque
        'max_depth': [None, 10, 20],  # Profundidad máxima de cada árbol
        'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para un split
        'min_samples_leaf': [1, 2],  # Mínimo número de muestras por hoja
        'bootstrap': [True, False]  # Si usar muestreo con reemplazo
    }
}

Definimos también otro diccionario con los modelos que vamos a utilizar.

In [49]:
# Modelos inicializados (sin parámetros)
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

Lanzamos cada uno de los modelos del diccionario anterior con su *Grid Search* correspondiente.

In [50]:
# Para cada modelo, realizar la búsqueda de hiperparámetros con GridSearchCV
best_models = {}
for name, model in models.items():
    print(f'Realizando GridSearch para {name}...')
    
    # GridSearchCV con validación cruzada (5 folds)
    grid_search = GridSearchCV(model, param_grids[name], cv=5, verbose=2, n_jobs=-1)
    
    # Entrenar usando el conjunto de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # Guardar el mejor modelo
    best_models[name] = grid_search.best_estimator_
    
    print(f'Mejores hiperparámetros para {name}: {grid_search.best_params_}')
    print('\n' + '-'*60 + '\n')

Realizando GridSearch para Logistic Regression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores hiperparámetros para Logistic Regression: {'C': 1, 'max_iter': 200, 'penalty': 'l2', 'solver': 'lbfgs'}

------------------------------------------------------------

Realizando GridSearch para Decision Tree...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Mejores hiperparámetros para Decision Tree: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

------------------------------------------------------------

Realizando GridSearch para Random Forest...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores hiperparámetros para Random Forest: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

------------------------------------------------------------



Evaluamos los mejores modelos

In [52]:
resultados = []

# Iterar sobre los modelos
for name, best_model in best_models.items():
    print(f'Evaluando {name}...')

    # Prediccion en el conjunto de test
    y_pred = best_model.predict(X_test)

    # Evaluar rendimiento
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Añadir los resultados a la lista
    resultados.append({
        'Modelo': name,
        'Accuracy': accuracy,
        'Matriz de Confusión': confusion,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    })

# Convertir la lista de resultados en un DataFrame de pandas
df_resultados = pd.DataFrame(resultados)

Evaluando Logistic Regression...
Evaluando Decision Tree...
Evaluando Random Forest...


In [53]:
df_resultados

,Modelo,Accuracy,Matriz de Confusión,Precision,Recall,F1-Score
0,Logistic Regression,0.767511,"[[694, 843], [176, 2670]]",0.773243,0.767511,0.747491
1,Decision Tree,0.671002,"[[1012, 525], [917, 1929]]",0.694384,0.671002,0.677440
2,Random Forest,0.761807,"[[726, 811], [233, 2613]]",0.761002,0.761807,0.745207


Por último, vamos a desarrollar una red neuronal que se encargue de hacer la predicción. En este caso vamos a enfocar el problema desde otra perspectiva, simplificando y optimizando la parte de preprocesado. En los modelos anteriores hemos empleado *One Hot Encoding*, generando un gran número de columnas. En este caso vamos a plantear la estandarización de dos formas diferentes:

- **denominacion** y **nombre_opositor**: vamos a utilizar embeddings para representar las cadenas de texto de forma numérica.
- **clase**: no hace falta tocar nada ya que se pueden introducir en las redes neuronales
- **resolucion**: lo dejamos como 1 y 0.

In [138]:
dataset_rn = df[['resolucion', 'denominacion', 'clase', 'nombre_opositor']]

In [139]:
dataset_rn.sample(5)

,resolucion,denominacion,clase,nombre_opositor
3125,aprobada_sin_oposicion,STA Satellite Max,[10],NaN
2179,aprobada_sin_oposicion,TAJ,[34],NaN
15005,negada_sin_oposicion,Gansulin,"[5, 10]",NaN
15267,negada_sin_oposicion,CPR Taylor,[9],NaN
1003,aprobada_sin_oposicion,LE PLIAGE,"[18, 25]",NaN


Eliminamos los valores nulos igual que hicimoz en el caso anterior y sustituimos los 'NaN', esta vez por un valor

In [140]:
# Eliminamos en denominacion
dataset_rn = dataset_rn.dropna(subset=['denominacion'])

# Sustituimos por 0 en nombre_opositor
dataset_rn['nombre_opositor'] = dataset_rn['nombre_opositor'].fillna('Sin Opositor')

In [141]:
# Comprobamos que se haya hecho bien
dataset_rn.isna().sum()

resolucion         0
denominacion       0
clase              0
nombre_opositor    0
dtype: int64

In [142]:
dataset_rn['resolucion'] = dataset_rn['resolucion'].replace({'aprobada_con_oposicion': 1, 'aprobada_sin_oposicion': 1, 
                                                             'negada_con_oposicion': 0, 'negada_sin_oposicion': 0
})

Codificamos las cadenas de texto de *denominacion* y *nombre_opositor*

In [143]:
denomination_le = LabelEncoder()
oppositor_le = LabelEncoder()

dataset_rn['denominacion'] = denomination_le.fit_transform(dataset_rn['denominacion'])
dataset_rn['nombre_opositor'] = oppositor_le.fit_transform(dataset_rn['nombre_opositor'])

Para la columna de clase vamos a utilizar el*MultiLabelBinzarized*

In [144]:
mlb = MultiLabelBinarizer()
dataset_rn_clase = mlb.fit_transform(dataset_rn['clase'])

A continuación, separamos la variable objetivo del resto del dataset y creamos ademas las variables predictoras

In [145]:
y = dataset_rn['resolucion']

X_denominacion = dataset_rn['denominacion']
X_opositor = dataset_rn['nombre_opositor']

Dividimos los datos en train y test

In [147]:
X_train_denom, X_test_denom, X_train_class, X_test_class, X_train_opo, X_test_opo, y_train, y_test = train_test_split(
    X_denominacion, dataset_rn_clase, X_opositor, y, test_size=0.3, random_state=1903)


Pasamos ahora a definir las entradas.

In [149]:
# Input para la denominación
input_denom = Input(shape=(1,), name='denominacion_input')

# Input para el nombre_opositor
input_opo = Input(shape=(1,), name='opositor_input')

# Input para las clases codificadas (binarias)
input_class = Input(shape=(dataset_rn_clase.shape[1],), name='class_input')

Y también las capas de embeddings

In [150]:
# Embeddings para denominación y nombre_opositor
embedding_denom = Embedding(input_dim=len(denomination_le.classes_), output_dim=10, name='denominacion_embedding')(input_denom)
embedding_opo = Embedding(input_dim=len(oppositor_le.classes_), output_dim=10, name='opositor_embedding')(input_opo)

# Convertimos los embeddings en un vector plano
flat_denom = Flatten()(embedding_denom)
flat_oppo = Flatten()(embedding_opo)

# Concatenamos los embeddings de denominación, opositor y las clases binarizadas
concat = Concatenate()([flat_denom, flat_oppo, input_class])

Añadimos las capas densas y la de salida

In [169]:
# Añadimos capas densas
dense_1 = Dense(64, activation='relu')(concat)
dense_2 = Dense(32, activation='relu')(dense_1)
dense_3 = Dense(16, activation='relu')(dense_2)

# Capa de salida (predicción binaria)
output = Dense(1, activation='sigmoid')(dense_3)


Por último, definimos y compilamos el modelo

In [170]:
# Definir el modelo completo
model = Model(inputs=[input_denom, input_opo, input_class], outputs=output)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [171]:
# Resumen del modelo
model.summary();

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ denominacion_input  │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ opositor_input      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ denominacion_embed… │ (None, 1, 10)     │    116,030 │ denominacion_inp… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ opositor_embedding  │ (None, 1, 10)     │     21,490 │ opositor_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_8 (Flatten) │ (None, 10)        │          0 │ denominacion_emb… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 10)        │          0 │ opositor_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_input         │ (None, 48)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 68)        │          0 │ flatten_8[0][0],  │
│ (Concatenate)       │                   │            │ flatten_9[0][0],  │
│                     │                   │            │ class_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 64)        │      4,416 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 32)        │      2,080 │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 16)        │        528 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 1)         │         17 │ dense_25[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 144,561 (564.69 KB)

 Trainable params: 144,561 (564.69 KB)

 Non-trainable params: 0 (0.00 B)

Ahora, entrenamos el modelo utilizando los datos de entrenamiento. Para las entradas, debemos proporcionar las variables separadas.

In [172]:
# Entrenar el modelo
history = model.fit(
    [X_train_denom, X_train_opo, X_train_class],  # Entradas
    y_train,  # Variable objetivo
    validation_data=([X_test_denom, X_test_opo, X_test_class], y_test), # Datos de validación
    epochs=250,  # Número de épocas
    batch_size=32  # Tamaño de lote
)


Epoch 1/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9606 - loss: 0.2324 - val_accuracy: 0.6496 - val_loss: 0.8047
Epoch 2/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9980 - loss: 0.0053 - val_accuracy: 0.6806 - val_loss: 0.7407
Epoch 3/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9977 - loss: 0.0043 - val_accuracy: 0.6703 - val_loss: 0.8690
Epoch 4/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9979 - loss: 0.0036 - val_accuracy: 0.6742 - val_loss: 0.8527
Epoch 5/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9985 - loss: 0.0033 - val_accuracy: 0.6452 - val_loss: 1.1791
Epoch 6/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9983 - loss: 0.0032 - val_accuracy: 0.6557 - val_loss: 1.0529
Epoch 7/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9981 - loss: 0.0037 - val_accuracy: 0.6740 - val_loss: 1.0282
Epoch 8/250
320/320 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9989 - loss: 0.0025 - val_accu

Por último pasamos a la evaluación del modelo

In [173]:
# Evaluar el modelo en el conjunto de test
loss, accuracy = model.evaluate([X_test_denom, X_test_opo, X_test_class], y_test)
print(f'Test Accuracy: {accuracy}')


137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.6571 - loss: 1.8193
Test Accuracy: 0.65571528673172
